### Hoodnaming

This notebook converts the point names downloaded from WebCTRL into a list of pseudo point names for later use

In [10]:
import pandas as pd
import numpy as np

In [11]:
url = 'https://github.com/thereidfleish/fume-hood-dashboard/blob/Robbie/report1678804296452.csv?raw=true'
pointnames = pd.read_csv(url)
pointnames.head()

,C1,Room,Pointname,Acronym,Point
0,Point List,NaN,NaN,NaN,NaN
1,Location:,Cornell University / Biotech Server / Biotech,NaN,NaN,NaN
2,report_progress_step(),NaN,NaN,NaN,NaN
3,report_progress_step(),NaN,NaN,NaN,NaN
4,report_progress_step(),NaN,NaN,NaN,NaN


In [12]:
#We want to collect the following:
#Air Flow Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log", HOOD FLOW

#Sash Opening Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash", HOOD SASH POS

#Occupancy Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ_trend", OCC SENSOR

#Internal Temperature Point, e.g: "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log", ZONE TEMP

import re
"""
The goal of this function is to output an organized collection of point names corresponding to specific fume hoods throughout the Biotechnology building. It is unclear the best path forward at the moment, however, we will try to start small and then expand upwards. There will probably be a lot of edge cases to handle, but we will see. Using regular expressions seems to have helped quite a bit.
"""


def csv_parser(df):
    #df['name_match'] = df['first_name'].apply(lambda x: 'Match' if x == 'Bill' else 'Mismatch')
    #print(True and df["Point"].str.contains("_control"))
    hood_flows = np.array(df[df["Pointname"]== "HOOD FLOW" ]["Point"])
    sash_pos = np.array(df[df["Pointname"]== "HOOD SASH POS"]["Point"])
    occ_sens = df[df["Pointname"]== "OCC SENSOR"]["Point"]
    occ_sens = [i for i in occ_sens if re.search("lab_[A-Za-z]*[0-9]+[A-Za-z]*_control",i)]
    zone_temps = df[df["Pointname"]== "ZONE TEMP"]["Point"]
    zone_temps = [i for i in zone_temps if re.search("lab_[A-Za-z]*[0-9]+[A-Za-z]*_control/zonetemp",i)]
    ##biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log
    zone_temps = [i.split("/")[:4] for i in zone_temps]
    zone_temps = ['/'.join(i) for i in zone_temps]
    zone_temps = [i + "/zone/zone_temp/trend_log" for i in zone_temps]

    vals= {"Hood":hood_flows, "Sash":sash_pos}
    odf = pd.DataFrame(data = vals)
    #display(ndf)
    # print(len(hood_flows))
    # print(len(sash_pos))
    # print(len(occ_sens))
    # print(len(zone_temps))

    ovals= {"Occupancy":occ_sens, "Internal Temp":zone_temps}
    ndf = pd.DataFrame(data = ovals)

    return [ndf,odf]
csv_parser(pointnames)[0]

,Occupancy,Internal Temp
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
...,...,...
65,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
66,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
67,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
68,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...


In [13]:
#Notice how the dataframes are not equal sizes; this is because certain labs are missing hood and sash points
#By looking at an excel file I found the following:
#Lab 147, Lab 217, Lab 218, Lab 427 are all missing hood and sash points
#We will remove these from the dataset

dfs = csv_parser(pointnames)
occ_temp = dfs[0]
hood_sash = dfs[1]

occ_temp = occ_temp.drop([3,9,11,58])
occ_temp = occ_temp.reset_index()
occ_temp = occ_temp.drop(["index"],axis=1)
occ_temp

,Occupancy,Internal Temp
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...


In [14]:
#Now we can just combine the dataframes together and we can begin working on a hierarchical structure

combined = hood_sash.join(occ_temp)
display(combined)

,Hood,Sash,Occupancy,Internal Temp
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...
...,...,...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...


In [15]:
def get_floor(text):
    blocks = text.split("/")
    floor_block = blocks[1]
    floor = floor_block.split("_")[1]
    return floor

combined['Floor'] = [get_floor(i) for i in combined["Hood"]]
#combined

def get_lab(text):
    blocks = text.split("/")
    lab_block = blocks[3]
    lab = lab_block.split("_")[1]
    return lab

combined["Lab"] = [get_lab(i) for i in combined["Hood"]]
combined

,Hood,Sash,Occupancy,Internal Temp,Floor,Lab
0,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,ground,g54
1,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1st,141
2,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1st,143
3,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1st,147b
4,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,1st,153
...,...,...,...,...,...,...
61,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4th,445b
62,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4th,453
63,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4th,453b
64,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,4th,457


In [16]:
#Rearranging the column orders :)


combined["External Temp"] = "#biotech/ground_flr_mech/building_hydronic_heating_syatems/reheat_heat_exchanger/oat"
combined["Server"] = "biotech_main"
cols = ["Lab","Floor","Server", "Hood","Sash","Occupancy","Internal Temp", "External Temp"]
combined = combined[cols]
#combined.to_csv('C:\\Users\\Rob\\Documents\\GitHub\\fume-hood-dashboard\\pointnames.csv')
combined

,Lab,Floor,Server,Hood,Sash,Occupancy,Internal Temp,External Temp
0,g54,ground,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...
1,141,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
2,143,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
3,147b,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
4,153,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...
...,...,...,...,...,...,...,...,...
61,445b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
62,453,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
63,453b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...
64,457,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...


In [7]:
df = pd.read_csv("pointnames.csv")
floor_map = {"ground":"Ground", "1st":"First","2nd":"Second", "3rd":"Third","4th":"Fourth"}
df["FloorNum"] = df["Floor"].map(floor_map)
display(df)

,Unnamed: 0,Lab,Floor,Server,Hood,Sash,Occupancy,Internal Temp,External Temp,FloorNum
0,0,g54,ground,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,Ground
1,1,141,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
2,2,143,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
3,3,147b,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
4,4,153,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First
...,...,...,...,...,...,...,...,...,...,...
61,61,445b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
62,62,453,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
63,63,453b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth
64,64,457,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth


In [5]:
#Biotech.floor_num.lab_num.hood_num.{pointType}
#4 Point Types:[energy.occ,energy.unocc,sashOpenTime.occ, sashOpenTime.unocc]

def point_maker(lab,floor,hood):
    nstr = "Biotech." + str(floor) + "Floor" + ".Lab_" +  str(lab)   + ".Hood_" + str(hood) + "."
    nlist = [nstr+"energy.occ",nstr+"energy.unocc",nstr+"sashOpenTime.occ",nstr+"sashOpenTime.unocc"]
    return nlist



#display(df)

# test_case = point_maker(141,1,1)
# test_case

def make_all_points(df):
    energy_occ = []
    energy_unocc = []
    sashOpenTime_occ = []
    sashOpenTime_unocc = []
    for i in range(len(df["FloorNum"])):
        response = point_maker(df["Lab"][i],df["FloorNum"][i],1)
        energy_occ.append(response[0])
        energy_unocc.append(response[1])
        sashOpenTime_occ.append(response[2])
        sashOpenTime_unocc.append(response[3])
    df["Energy Occupied"] = energy_occ
    df["Energy Unoccupied"] = energy_unocc
    df["Sash Open Time Occupied"] = sashOpenTime_occ
    df["Sash Open Time Unoccupied"] = sashOpenTime_unocc
    return df

ndf = make_all_points(df)
display(ndf)







,Unnamed: 0,Lab,Floor,Server,Hood,Sash,Occupancy,Internal Temp,External Temp,FloorNum,Energy Occupied,Energy Unoccupied,Sash Open Time Occupied,Sash Open Time Unoccupied
0,0,g54,ground,biotech_main,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/biotech_ground_floor/ground_floor_fum...,#biotech/ground_flr_mech/building_hydronic_hea...,Ground,Biotech.GroundFloor.Lab_g54.Hood_1.energy.occ,Biotech.GroundFloor.Lab_g54.Hood_1.energy.unocc,Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTim...,Biotech.GroundFloor.Lab_g54.Hood_1.sashOpenTim...
1,1,141,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_141.Hood_1.energy.occ,Biotech.FirstFloor.Lab_141.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_141.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_141.Hood_1.sashOpenTime...
2,2,143,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_143.Hood_1.energy.occ,Biotech.FirstFloor.Lab_143.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_143.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_143.Hood_1.sashOpenTime...
3,3,147b,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_147b.Hood_1.energy.occ,Biotech.FirstFloor.Lab_147b.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_147b.Hood_1.sashOpenTim...,Biotech.FirstFloor.Lab_147b.Hood_1.sashOpenTim...
4,4,153,1st,biotech_main,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/biotech_1st_floor/first_floor_fume_ho...,#biotech/ground_flr_mech/building_hydronic_hea...,First,Biotech.FirstFloor.Lab_153.Hood_1.energy.occ,Biotech.FirstFloor.Lab_153.Hood_1.energy.unocc,Biotech.FirstFloor.Lab_153.Hood_1.sashOpenTime...,Biotech.FirstFloor.Lab_153.Hood_1.sashOpenTime...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,445b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_445b.Hood_1.energy.occ,Biotech.FourthFloor.Lab_445b.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_445b.Hood_1.sashOpenTi...,Biotech.FourthFloor.Lab_445b.Hood_1.sashOpenTi...
62,62,453,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_453.Hood_1.energy.occ,Biotech.FourthFloor.Lab_453.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_453.Hood_1.sashOpenTim...,Biotech.FourthFloor.Lab_453.Hood_1.sashOpenTim...
63,63,453b,4th,biotech_main,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/biotech_4th_floor/fourth_floor_fume_h...,#biotech/ground_flr_mech/building_hydronic_hea...,Fourth,Biotech.FourthFloor.Lab_453b.Hood_1.energy.occ,Biotech.FourthFloor.Lab_453b.Hood_1.energy.unocc,Biotech.FourthFloor.Lab_453b.Hood_1.sashOpenTi...,Biotec

In [6]:
ndf.to_csv("namingconvention.csv")